<a href="https://colab.research.google.com/github/Sharathkumar245/DL_Assignment_1/blob/main/partA/dl_assignment_partA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchvision
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
data_folder="/content/drive/MyDrive/inaturalist_12K/train"

In [ ]:
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
transform=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])
naturalist_dataset=ImageFolder(root=data_folder,transform=transform)
train_data_size=int(0.8*(len(naturalist_dataset)))
valid_data_size=int(len(naturalist_dataset)-train_data_size)
train_dataset,valid_dataset=torch.utils.data.random_split(naturalist_dataset,[train_data_size,valid_data_size])
train_loader=data.DataLoader(train_dataset,batch_size=64,shuffle=True)
valid_loader=data.DataLoader(valid_dataset,batch_size=64,shuffle=False)



# print("No of classes: ",len(naturalist_dataset.classes))
# print("No of training images: ",len(train_dataset))
# print("No of validation images: ",len(valid_dataset))
# sample_image,sample_label=train_dataset[0]
# print(sample_image.shape)
# naturalist_dataset


#define a CNN model
class convnetwork(nn.Module):
  def __init__(self,in_channels,num_classes,conv_filters,conv_filter_sizes,conv_strides,
              pool_filter_sizes,pool_strides,hidden_units,activation_func=nn.ReLU):
    super(convnetwork,self).__init__()
    #create an array to store the layers
    self.conv_layers=nn.ModuleList()
    self.pool_layers=nn.ModuleList()
    self.activations=nn.ModuleList()
    for i in range(len(conv_filters)):
      conv_layer=nn.Conv2d(in_channels,conv_filters[i],conv_filter_sizes[i],conv_strides[i],1)
      activation_layer=activation_func()
      pool_layer=nn.MaxPool2d(pool_filter_sizes[i],pool_strides[i])

      #append these layers to the list that we created lists
      self.conv_layers.append(conv_layer)
      self.activations.append(activation_layer)
      self.pool_layers.append(pool_layer)

      in_channels=conv_filters[i];
    self.fc=nn.Linear(12544,hidden_units)
    self.output_layer=nn.Linear(hidden_units,num_classes)

    #now do the forward propagation on the layers in the network
  def forward(self,x):
    for i in range(len(self.conv_layers)):
      x=self.conv_layers[i](x)
      x=self.activations[i](x)
      x=self.pool_layers[i](x)

    x=x.view(x.size(0),-1)
    x=self.fc(x)
    x=self.output_layer(x)
    return x

  #we need the configuration for the class so we are giving those
model = convnetwork(
    in_channels=3,
    num_classes=10,
    conv_filters=[16,32,64,128,256],
    conv_filter_sizes=[3,3,3,3,3],
    conv_strides=[1,1,1,1,1],
    pool_filter_sizes=[2,2,2,2,2],
    pool_strides=[2,2,2,2,2],
    hidden_units=256,
    activation_func=nn.ReLU
)

random_input=torch.randn(1,3,224,224)
output=model(random_input)
print(output.shape)

torch.Size([1, 10])
